In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark2 =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
  .getOrCreate() 
sqlContext = SQLContext(spark2)

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5103a17b-68a2-4d77-a87a-35e7051511bc;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 in central
:: resolution report :: resolve 339ms :: artifacts dl 4ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrie

In [2]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/nft_sales.csv")
df.show(5)

+--------------------+--------------+---------+----------+---------+
|         Collections|         Sales|   Buyers|      Txns|   Owners|
+--------------------+--------------+---------+----------+---------+
|       Axie Infinity|$3,328,148,500|1,079,811| 9,755,511|2,656,431|
|         CryptoPunks|$1,664,246,968|    4,723|    18,961|    3,289|
|          Art Blocks|$1,075,223,906|   20,934|   117,602|   25,094|
|Bored Ape Yacht Club|  $783,882,186|    8,284|    22,584|    5,862|
|        NBA Top Shot|  $781,965,423|  374,818|11,790,699|  603,928|
+--------------------+--------------+---------+----------+---------+
only showing top 5 rows



In [3]:
df.count()

250

In [4]:
df.printSchema()

root
 |-- Collections: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Buyers: string (nullable = true)
 |-- Txns: string (nullable = true)
 |-- Owners: string (nullable = true)



Find NA values in this table. Only Owners are Na, so changing the value to -1
-----------------------------------------

In [5]:
df.filter(df.Sales.isNull()).show()
df.filter(df.Collections.isNull()).show()
df.filter(df.Buyers.isNull()).show()
df.filter(df.Txns.isNull()).show()
df.filter(df.Owners.isNull()).show()

+-----------+-----+------+----+------+
|Collections|Sales|Buyers|Txns|Owners|
+-----------+-----+------+----+------+
+-----------+-----+------+----+------+

+-----------+-----+------+----+------+
|Collections|Sales|Buyers|Txns|Owners|
+-----------+-----+------+----+------+
+-----------+-----+------+----+------+

+-----------+-----+------+----+------+
|Collections|Sales|Buyers|Txns|Owners|
+-----------+-----+------+----+------+
+-----------+-----+------+----+------+

+-----------+-----+------+----+------+
|Collections|Sales|Buyers|Txns|Owners|
+-----------+-----+------+----+------+
+-----------+-----+------+----+------+

+------------------+------------+------+------+------+
|       Collections|       Sales|Buyers|  Txns|Owners|
+------------------+------------+------+------+------+
|    Parallel Alpha|$163,724,921|11,103|67,736|  null|
|   Gutter Cat Gang| $35,876,258| 1,729| 3,343|  null|
|     Frontier Game| $23,972,975| 3,257| 7,409|  null|
|       Gutter Rats| $12,682,958| 1,738| 3

In [6]:
df = df.withColumnRenamed("Sales","SalesinUSD")
df = df.withColumnRenamed("Txns","Transactions")
# df.printSchema()
df = df.withColumn("SalesinUSD",regexp_replace(col("SalesinUSD"), '[$,]', ""))
df = df.withColumn("Buyers",regexp_replace(col("Buyers"), ",", ""))
df = df.withColumn("Transactions",regexp_replace(col("Transactions"), ",", ""))
df = df.withColumn("Owners",regexp_replace(col("Owners"), ",", ""))
# df=df.dropna()
df.show()

+--------------------+----------+-------+------------+-------+
|         Collections|SalesinUSD| Buyers|Transactions| Owners|
+--------------------+----------+-------+------------+-------+
|       Axie Infinity|3328148500|1079811|     9755511|2656431|
|         CryptoPunks|1664246968|   4723|       18961|   3289|
|          Art Blocks|1075223906|  20934|      117602|  25094|
|Bored Ape Yacht Club| 783882186|   8284|       22584|   5862|
|        NBA Top Shot| 781965423| 374818|    11790699| 603928|
|Mutant Ape Yacht ...| 422429206|  10350|       17343|  10254|
|                Loot| 264652035|   3571|        9997|   2555|
|             Meebits| 237519388|   5112|       13580|   5361|
|           Cool Cats| 186937798|   7960|       23007|   5029|
|           CrypToadz| 170407439|   6277|       15005|   3525|
|      Parallel Alpha| 163724921|  11103|       67736|   null|
|           MekaVerse| 157475681|   4668|        9335|   5089|
|           0N1 Force| 147262453|   6917|       15982| 

In [7]:
df = df.withColumn("SalesinUSD", df["SalesinUSD"].cast(LongType()))
df = df.withColumn("Buyers", df["Buyers"].cast(IntegerType()))
df = df.withColumn("Transactions", df["Transactions"].cast(IntegerType()))
df = df.withColumn("Owners", df["Owners"].cast(IntegerType()))
df.show(5)
df.printSchema()

+--------------------+----------+-------+------------+-------+
|         Collections|SalesinUSD| Buyers|Transactions| Owners|
+--------------------+----------+-------+------------+-------+
|       Axie Infinity|3328148500|1079811|     9755511|2656431|
|         CryptoPunks|1664246968|   4723|       18961|   3289|
|          Art Blocks|1075223906|  20934|      117602|  25094|
|Bored Ape Yacht Club| 783882186|   8284|       22584|   5862|
|        NBA Top Shot| 781965423| 374818|    11790699| 603928|
+--------------------+----------+-------+------------+-------+
only showing top 5 rows

root
 |-- Collections: string (nullable = true)
 |-- SalesinUSD: long (nullable = true)
 |-- Buyers: integer (nullable = true)
 |-- Transactions: integer (nullable = true)
 |-- Owners: integer (nullable = true)



In [8]:
df = df.na.fill(-1)
df = df.withColumn('Hundredmillclub', when(col('SalesinUSD') > 100000000, "Yes").otherwise("No"))
df.show(5)

+--------------------+----------+-------+------------+-------+---------------+
|         Collections|SalesinUSD| Buyers|Transactions| Owners|Hundredmillclub|
+--------------------+----------+-------+------------+-------+---------------+
|       Axie Infinity|3328148500|1079811|     9755511|2656431|            Yes|
|         CryptoPunks|1664246968|   4723|       18961|   3289|            Yes|
|          Art Blocks|1075223906|  20934|      117602|  25094|            Yes|
|Bored Ape Yacht Club| 783882186|   8284|       22584|   5862|            Yes|
|        NBA Top Shot| 781965423| 374818|    11790699| 603928|            Yes|
+--------------------+----------+-------+------------+-------+---------------+
only showing top 5 rows



In [12]:
df.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'Top250_nftdata.etl_output').mode('overwrite').save()